# Preprocess test dataset

In [1]:
import pandas as pd
import numpy as np

#Math tools
from scipy import stats
from scipy.stats import skew,norm  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
import scipy.stats as stats

In [14]:
train = pd.read_csv('./raw_data/train.csv')
test = pd.read_csv('./raw_data/test.csv')

In [15]:
train = train.drop('SalePrice', axis=1)

In [19]:
df = pd.concat([train, test])

In [20]:
print("Total No. of missing value {} before Imputation".format(sum(df.isnull().sum())))

class missing_values():
    
    for col_cat in ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'GarageCond', 'GarageQual', 'GarageYrBlt',
               'GarageFinish', 'GarageType', 'BsmtCond', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
               'MasVnrType', 'MSSubClass']:
        df[col_cat] = df[col_cat].fillna('None')
    
    # LotFrontage : Since the area of each street connected to the house property most likely have a similar area to 
    # other houses in its neighborhood , we can fill in missing values by the median LotFrontage of the neighborhood.
    #Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
    df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
    df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])
    # MSZoning (The general zoning classification) : 'RL' is by far the most common value. So we can fill in 
    # missing values with 'RL'
    df['MSZoning'] = df['MSZoning'].fillna(df['MSZoning'].mode()[0])
    # Electrical : It has one NA value. Since this feature has mostly 'SBrkr', we can set that for the missing value.
    df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])
    # Functional : data description says NA means typical
    df["Functional"] = df["Functional"].fillna("Typ")
    # KitchenQual: Only one NA value, and same as Electrical, we set 'TA' (which is the most frequent) for the missing 
    # value in KitchenQual.
    df['KitchenQual'] = df['KitchenQual'].fillna(df['KitchenQual'].mode()[0])
    # Exterior1st and Exterior2nd : Again Both Exterior 1 & 2 have only one missing value. We will just substitute in 
    # the most common string
    df['Exterior1st'] = df['Exterior1st'].fillna(df['Exterior1st'].mode()[0])
    df['Exterior2nd'] = df['Exterior2nd'].fillna(df['Exterior2nd'].mode()[0])
    # SaleType : Fill in again with most frequent which is "WD"
    df['SaleType'] = df['SaleType'].fillna(df['SaleType'].mode()[0])
     
    df['Utilities'] = df['Utilities'].fillna(df['Utilities'].mode()[0])
    df['GarageCars'] = df['GarageCars'].fillna(df['GarageCars'].mode()[0])
    df['GarageArea'] = df['GarageArea'].fillna(df['GarageArea'].mode()[0])
    

    for col_num in ['MasVnrArea', 'BsmtHalfBath', 'BsmtFullBath', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFinSF2', 'BsmtFinSF1']:
        df[col_num] = df[col_num].fillna(0)
    
missing_values()

print("Total No. of missing value {} after Imputation".format(sum(df.isnull().sum())))

Total No. of missing value 13965 before Imputation
Total No. of missing value 0 after Imputation


In [21]:
# converting some numeric features to string

#MSSubClass: The building class
df['MSSubClass'] = df['MSSubClass'].apply(str)
# YrSold: Year Sold
df['YrSold'] = df['YrSold'].astype(str)
#MoSold: Month Sold
df['MoSold'] = df['MoSold'].astype(str)
# GarageYrBlt: Year garage was built
df['GarageYrBlt'] = df['GarageYrBlt'].astype(str)

In [22]:
# YrSold: Year Sold Original construction date + YearRemodAdd: Remodel date
df['YearsSinceRemodel'] = df['YrSold'].astype(int) + df['YearRemodAdd'].astype(int)
# OverallQual: Overall material and finish quality + OverallCond: Overall condition rating
df['Total_Home_Quality'] = df['OverallQual'] + df['OverallCond']
# TotalBsmtSF: Total square feet of basement area + 1stFlrSF: First Floor square feet + 2ndFlrSF: Second floor square feet
df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
# YearBuilt: Original construction date + YearRemodAdd: Remodel date
df['YrBltAndRemod'] = df['YearBuilt'] + df['YearRemodAdd']
# BsmtFinSF1: Type 1 finished square feet + BsmtFinSF2: Type 2 finished square feet + 
# 1stFlrSF: First Floor square feet + 2ndFlrSF: Second floor square feet
df['Total_sqr_footage'] = (df['BsmtFinSF1'] + df['BsmtFinSF2'] + df['1stFlrSF'] + df['2ndFlrSF'])
# FullBath: Full bathrooms above grade + HalfBath: Half baths above grade + 
# BsmtFullBath: Basement full bathrooms + BsmtHalfBath: Basement half bathrooms
df['Total_Bathrooms'] = (df['FullBath'] + (0.5 * df['HalfBath']) + df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath']))
# OpenPorchSF: Open porch area in square feet + 3SsnPorch: Three season porch area in square feet +
# EnclosedPorch: Enclosed porch area in square feet + ScreenPorch: Screen porch area in square feet + 
# WoodDeckSF: Wood deck area in square feet
df['Total_porch_sf'] = (df['OpenPorchSF'] + df['3SsnPorch'] + df['EnclosedPorch'] + df['ScreenPorch'] + df['WoodDeckSF'])

In [23]:
# simplified features
df['haspool'] = df['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
df['has2ndfloor'] = df['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
df['hasgarage'] = df['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
df['hasbsmt'] = df['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
df['hasfireplace'] = df['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [24]:
# Filter the skewed features
numeric = df.select_dtypes(include='number').columns
skew_features = df[numeric].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

print("There are {} numerical features with Skew > 0.5 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skew_features.head(10)

There are 29 numerical features with Skew > 0.5 :


MiscVal          21.947195
PoolArea         16.898328
haspool          14.884318
LotArea          12.822431
LowQualFinSF     12.088761
3SsnPorch        11.376065
KitchenAbvGr      4.302254
BsmtFinSF2        4.146143
EnclosedPorch     4.003891
ScreenPorch       3.946694
dtype: float64

In [25]:
# Normalize skewed features using boxcox
for i in skew_index:
    df[i] = boxcox1p(df[i], boxcox_normmax(df[i] + 1))

In [26]:
# Get dummies
df = pd.get_dummies(df, prefix_sep='_', drop_first=True)

In [27]:
overfitted_features = ['PoolArea',
 'haspool',
 'MSSubClass_180',
 'MSSubClass_40',
 'MSSubClass_45',
 'Street_Pave',
 'LotShape_IR3',
 'Utilities_NoSeWa',
 'LotConfig_FR3',
 'LandSlope_Sev',
 'Neighborhood_Blueste',
 'Neighborhood_NPkVill',
 'Neighborhood_Veenker',
 'Condition1_PosA',
 'Condition1_RRAe',
 'Condition1_RRNe',
 'Condition1_RRNn',
 'Condition2_Feedr',
 'Condition2_PosA',
 'Condition2_PosN',
 'Condition2_RRAe',
 'Condition2_RRAn',
 'Condition2_RRNn',
 'HouseStyle_1.5Unf',
 'HouseStyle_2.5Fin',
 'HouseStyle_2.5Unf',
 'RoofStyle_Gambrel',
 'RoofStyle_Mansard',
 'RoofStyle_Shed',
 'RoofMatl_Membran',
 'RoofMatl_Metal',
 'RoofMatl_Roll',
 'RoofMatl_Tar&Grv',
 'RoofMatl_WdShake',
 'RoofMatl_WdShngl',
 'Exterior1st_AsphShn',
 'Exterior1st_BrkComm',
 'Exterior1st_CBlock',
 'Exterior1st_ImStucc',
 'Exterior1st_Stone',
 'Exterior2nd_AsphShn',
 'Exterior2nd_Brk Cmn',
 'Exterior2nd_CBlock',
 'Exterior2nd_ImStucc',
 'Exterior2nd_Other',
 'Exterior2nd_Stone',
 'ExterQual_Fa',
 'ExterCond_Po',
 'Foundation_Stone',
 'Foundation_Wood',
 'BsmtCond_Po',
 'BsmtFinType2_GLQ',
 'Heating_Grav',
 'Heating_OthW',
 'Heating_Wall',
 'HeatingQC_Po',
 'Electrical_FuseP',
 'Electrical_Mix',
 'Functional_Maj2',
 'Functional_Sev',
 'GarageType_CarPort',
 'GarageYrBlt_1906.0',
 'GarageYrBlt_1908.0',
 'GarageYrBlt_1910.0',
 'GarageYrBlt_1914.0',
 'GarageYrBlt_1915.0',
 'GarageYrBlt_1916.0',
 'GarageYrBlt_1918.0',
 'GarageYrBlt_1920.0',
 'GarageYrBlt_1921.0',
 'GarageYrBlt_1922.0',
 'GarageYrBlt_1923.0',
 'GarageYrBlt_1924.0',
 'GarageYrBlt_1925.0',
 'GarageYrBlt_1926.0',
 'GarageYrBlt_1927.0',
 'GarageYrBlt_1928.0',
 'GarageYrBlt_1929.0',
 'GarageYrBlt_1930.0',
 'GarageYrBlt_1931.0',
 'GarageYrBlt_1932.0',
 'GarageYrBlt_1933.0',
 'GarageYrBlt_1934.0',
 'GarageYrBlt_1935.0',
 'GarageYrBlt_1936.0',
 'GarageYrBlt_1937.0',
 'GarageYrBlt_1938.0',
 'GarageYrBlt_1939.0',
 'GarageYrBlt_1940.0',
 'GarageYrBlt_1941.0',
 'GarageYrBlt_1942.0',
 'GarageYrBlt_1945.0',
 'GarageYrBlt_1946.0',
 'GarageYrBlt_1947.0',
 'GarageYrBlt_1948.0',
 'GarageYrBlt_1949.0',
 'GarageYrBlt_1951.0',
 'GarageYrBlt_1952.0',
 'GarageYrBlt_1953.0',
 'GarageYrBlt_1955.0',
 'GarageYrBlt_1961.0',
 'GarageYrBlt_1971.0',
 'GarageYrBlt_1972.0',
 'GarageYrBlt_1973.0',
 'GarageYrBlt_1975.0',
 'GarageYrBlt_1981.0',
 'GarageYrBlt_1982.0',
 'GarageYrBlt_1983.0',
 'GarageYrBlt_1984.0',
 'GarageYrBlt_1985.0',
 'GarageYrBlt_1986.0',
 'GarageYrBlt_1987.0',
 'GarageYrBlt_1988.0',
 'GarageYrBlt_1989.0',
 'GarageYrBlt_1991.0',
 'GarageYrBlt_1992.0',
 'GarageYrBlt_2010.0',
 'GarageQual_Gd',
 'GarageQual_Po',
 'GarageCond_Gd',
 'GarageCond_Po',
 'PoolQC_Fa',
 'PoolQC_Gd',
 'PoolQC_None',
 'Fence_MnWw',
 'MiscFeature_Othr',
 'MiscFeature_TenC',
 'SaleType_CWD',
 'SaleType_Con',
 'SaleType_ConLD',
 'SaleType_ConLI',
 'SaleType_ConLw',
 'SaleType_Oth',
 'SaleCondition_AdjLand',
 'SaleCondition_Alloca']

In [28]:
overfitted_features

['PoolArea',
 'haspool',
 'MSSubClass_180',
 'MSSubClass_40',
 'MSSubClass_45',
 'Street_Pave',
 'LotShape_IR3',
 'Utilities_NoSeWa',
 'LotConfig_FR3',
 'LandSlope_Sev',
 'Neighborhood_Blueste',
 'Neighborhood_NPkVill',
 'Neighborhood_Veenker',
 'Condition1_PosA',
 'Condition1_RRAe',
 'Condition1_RRNe',
 'Condition1_RRNn',
 'Condition2_Feedr',
 'Condition2_PosA',
 'Condition2_PosN',
 'Condition2_RRAe',
 'Condition2_RRAn',
 'Condition2_RRNn',
 'HouseStyle_1.5Unf',
 'HouseStyle_2.5Fin',
 'HouseStyle_2.5Unf',
 'RoofStyle_Gambrel',
 'RoofStyle_Mansard',
 'RoofStyle_Shed',
 'RoofMatl_Membran',
 'RoofMatl_Metal',
 'RoofMatl_Roll',
 'RoofMatl_Tar&Grv',
 'RoofMatl_WdShake',
 'RoofMatl_WdShngl',
 'Exterior1st_AsphShn',
 'Exterior1st_BrkComm',
 'Exterior1st_CBlock',
 'Exterior1st_ImStucc',
 'Exterior1st_Stone',
 'Exterior2nd_AsphShn',
 'Exterior2nd_Brk Cmn',
 'Exterior2nd_CBlock',
 'Exterior2nd_ImStucc',
 'Exterior2nd_Other',
 'Exterior2nd_Stone',
 'ExterQual_Fa',
 'ExterCond_Po',
 'Foundation_S

In [29]:
df.drop(overfitted_features, inplace=True, axis=1)

In [30]:
df.shape

(2919, 266)

In [31]:
df = df.iloc[len(train):, :]

In [45]:
col = pd.DataFrame(df.columns)

In [43]:
col.head(2)

,0
0,Id
1,LotFrontage


In [37]:
df_2 = pd.read_csv('preprossed_data/preprossed_data.csv')

In [40]:
col_2 = pd.DataFrame(df_2.columns)

In [49]:
col_2.head(2)

,0
0,LotFrontage
1,LotArea


In [50]:
result  = col.merge(col_2, how='outer', indicator=True).loc[lambda x : x['_merge']=='left_only']

In [51]:
result

,0,_merge
0,Id,left_only
43,MSSubClass_150,left_only
183,GarageYrBlt_1896.0,left_only
184,GarageYrBlt_1900.0,left_only
185,GarageYrBlt_1917.0,left_only
186,GarageYrBlt_1919.0,left_only
187,GarageYrBlt_1943.0,left_only
228,GarageYrBlt_2207.0,left_only


In [53]:
df.drop(['Id', 'MSSubClass_150', 'GarageYrBlt_1896.0', 'GarageYrBlt_1900.0', 'GarageYrBlt_1917.0', 'GarageYrBlt_1919.0',
         'GarageYrBlt_1943.0', 'GarageYrBlt_2207.0'], inplace=True, axis=1)

In [54]:
df.shape

(1459, 258)

In [55]:
df.to_csv('preprossed_data/preprossed_test.csv', index=False)